In [13]:
import random
from 有用なスクリプト import load_gemini

In [14]:
model, tokenizer = load_gemini()

In [15]:
import torch
messages = [
    [
        {
            "role": "user",
            "content": [{"type": "text", "text": "**一言というと**、LLMは何ですか？"},]
        },
    ],
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(model.device)


with torch.inference_mode():
    outputs = model.generate(**inputs, max_new_tokens=64, temperature = 1.3)

responses = tokenizer.batch_decode(outputs)
print(responses[0])

<bos><start_of_turn>user
**一言というと**、LLMは何ですか？<end_of_turn>
<start_of_turn>model
大規模言語モデルです。
<end_of_turn>


In [62]:
def get_unique_random_numbers(start, stop, n, seed = 42):
  """startからstopまでの区間の中に、乱数的にn個の整数を選ぶ
  """
  population_size = stop - start
  if n > population_size:
    raise ValueError(f"区間の長さ{population_size}はn ({n})以下")
  
  # ここでseedを設置すると、何度この関数を回すでも、結果は変わらないようにする
  random.seed(seed)
  return sorted(random.sample(range(start, stop), n))

In [63]:
random_list_1 = get_unique_random_numbers(0, 1000, 5, seed = 114)
random_list_2 = get_unique_random_numbers(0, 1000, 5, seed = 514)
print("リスト1:" ,random_list_1)
print("リスト2:" ,random_list_2)

リスト1: [100, 246, 344, 576, 681]
リスト2: [112, 213, 366, 647, 669]


In [64]:
green_red_list_ratio = 0.5

meaningful_start_at = 237
meaningful_end_at = 255968

len_meaningful = meaningful_end_at - meaningful_start_at

len_green_list = int(green_red_list_ratio * len_meaningful)
print(f"トークンの総数：{tokenizer.vocab_size}")
print(f"意味のあるトークンの総数：{len_meaningful}")
print(f"そのうち緑のトークンの数は{len_green_list}")

トークンの総数：262144
意味のあるトークンの総数：255731
そのうち緑のトークンの数は127865


In [70]:
green_ids = get_unique_random_numbers(
    start = meaningful_start_at,
    stop  = meaningful_end_at, 
    n = len_green_list
)
print(green_ids[:10])

[237, 239, 246, 250, 251, 252, 254, 255, 257, 259]


In [71]:

color_choice = ["red", "green"] 
word_list = list()
color_list = list()

for token in outputs.tolist()[0]:
    word = tokenizer.decode(token)
    color = color_choice[token in green_ids]
    
    word_list.append(word)
    color_list.append(color)



In [20]:
from rich.console import Console
from rich.panel import Panel
from rich.text import Text
from rich import box
# FIX: Import a valid theme object
from rich.terminal_theme import MONOKAI , SVG_EXPORT_THEME
from rich.terminal_theme import TerminalTheme
LIGHT_MONOKAI = TerminalTheme(
    (255, 255, 255),     # light background
    (40, 40, 40),        # dark foreground (Monokai-like)
    [
        (240, 240, 240),  # black -> light gray
        (244, 0, 95),      # red
        (152, 224, 36),    # green
        (253, 151, 31),    # yellow/orange
        (102, 102, 255),   # blue-ish (slightly darker for readability)
        (244, 0, 95),      # magenta
        (88, 209, 235),    # cyan
        (60, 60, 60),      # white -> dark gray
    ],
    [
        (128, 128, 128),
        (255, 0, 95),
        (0, 200, 0),
        (255, 200, 0),
        (0, 0, 200),
        (255, 0, 255),
        (0, 200, 200),
        (20, 20, 20),
    ],
)


def save_colored_box(words, colors, filename="output.svg", title="Output"):
    # 1. Initialize Console with 'record=True' and a set width
    console = Console(record=True, width=100) 

    # 2. Construct the content
    styled_content = Text()
    for word, color in zip(words, colors):
        styled_content.append(word, style=color)


    # 3. Create the Panel
    panel = Panel(
        styled_content,
        # title=title,
        # box=box.ROUNDED,
        # padding=(1, 2),
        # expand=False
    )

    # 4. Print to the virtual console
    console.print(panel)

    # 5. Save using the correct theme object
    # MONOKAI provides a clean dark background automatically.
    # If you want the default terminal look, just remove the 'theme' argument entirely.
    console.save_svg(filename, title=title, theme=SVG_EXPORT_THEME)
    # print(f"Successfully saved to {filename}")


In [ ]:

color_choice = ["red", "green"] 
word_list = list()
color_list = list()

for token in outputs.tolist()[0]:
    word = tokenizer.decode(token)
    color = color_choice[token in green_ids_1]
    
    word_list.append(word)
    color_list.append(color)

save_colored_box(word_list, color_list, filename="my_design.svg")

In [31]:
from rich.console import Console
from rich.panel import Panel
from rich.text import Text
from rich import box
from rich.terminal_theme import MONOKAI

def print_box(words, colors, filename="", title="出力結果"):
    # 1. Initialize Console
    # width=40 is usually enough for Japanese (since chars are wide)
    console = Console(record=True, width=75) 

    styled_content = Text()
    for word, color in zip(words, colors):
        for w in word:
            styled_content.append(w, style=color)


    panel = Panel(
        styled_content,
        # title=title,
        box=box.ROUNDED,
        padding=(1, 2),
        expand=True
    )

    console.print(panel)

    if filename:
        console.save_svg(
            filename, 
            title=title, 
            theme=LIGHT_MONOKAI,
            # font_family="MS Gothic, Hiragino Sans, AppleGothic, Noto Sans CJK JP, monospace",
        )
        print(f"Saved to {filename}")

# --- Usage with Japanese ---
word_list = ["これ", "は", "Python", "の", "Rich", "ライブラリ", "の", "テスト", "です。"]
color_list = ["cyan", "white", "green", "white", "magenta", "yellow", "white", "red", "white"]

print_box(word_list, color_list)

╭─────────────────────────────────────────────────────────────────────────╮
│                                                                         │
│  これはPythonのRichライブラリのテストです。                             │
│                                                                         │
╰─────────────────────────────────────────────────────────────────────────╯

In [73]:
messages = [
    [
        {
            "role": "user",
            "content": [{"type": "text", "text": "**一言というと**、LLMは何ですか？"},]
        },
    ],
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(model.device)


with torch.inference_mode():
    outputs = model.generate(**inputs, max_new_tokens=64, temperature = 1.3)

responses = tokenizer.batch_decode(outputs)
print(responses[0])

<bos><start_of_turn>user
**一言というと**、LLMは何ですか？<end_of_turn>
<start_of_turn>model
大規模言語モデル（LLM）とは、大量のテキストデータを学習し、人間が書いたようなテキストを生成することができるAIのモデルのことです。
<end_of_turn>


In [ ]:
color_choice = ["red", "green4"] 
word_list = list()
color_list = list()

for token in outputs.tolist()[0]:
    word = tokenizer.decode(token)
    color = color_choice[token in green_ids]
    
    word_list.append(word)
    color_list.append(color)

print_box(word_list, color_list, filename="../pics/simple_vis.svg", title="可視化")

╭─────────────────────────────────────────────────────────────────────────╮
│                                                                         │
│  <bos><start_of_turn>user                                               │
│  **一言というと**、LLMは何ですか？<end_of_turn>                         │
│  <start_of_turn>model                                                   │
│  大規模言語モデル（LLM）とは、大量のテキストデータを学習し、人間が書い  │
│  たようなテキストを生成することができるAIのモデルのことです。           │
│  <end_of_turn>                                                          │
│                                                                         │
╰─────────────────────────────────────────────────────────────────────────╯

Saved to ../pics/simple_vis.svg
